<a href="https://www.kaggle.com/code/rahulks14322/predicting-accident-risk?scriptVersionId=269668604" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [166]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
Sample=pd.read_csv("/kaggle/input/playground-series-s5e10/sample_submission.csv")
test=pd.read_csv("/kaggle/input/playground-series-s5e10/test.csv")
train=pd.read_csv("/kaggle/input/playground-series-s5e10/train.csv")

In [167]:
## The data of train is ##
train.head()

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56


In [168]:
## The data of test is ##
test.head()

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents
0,517754,highway,2,0.34,45,night,clear,True,True,afternoon,True,True,1
1,517755,urban,3,0.04,45,dim,foggy,True,False,afternoon,True,False,0
2,517756,urban,2,0.59,35,dim,clear,True,False,afternoon,True,True,1
3,517757,rural,4,0.95,35,daylight,rainy,False,False,afternoon,False,False,2
4,517758,highway,2,0.86,35,daylight,clear,True,False,evening,False,True,3


In [169]:
## The data of Sample is ##
Sample.head()

,id,accident_risk
0,517754,0.352
1,517755,0.352
2,517756,0.352
3,517757,0.352
4,517758,0.352


In [170]:
## clenning the data ##
train.head()#i remove the id column..


,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56


In [171]:
## creating the feature for the model ##

X=train.drop(columns=['id','accident_risk'])
y=train['accident_risk']
X_test = test.drop(columns=['id'], axis=1)

In [172]:
# Define feature types
categorical = ['road_type', 'lighting', 'weather', 'time_of_day']
boolean = ['road_signs_present', 'public_road', 'holiday', 'school_season']
numeric = ['num_lanes', 'curvature', 'speed_limit', 'num_reported_accidents']

In [173]:
#Preprocessor
preprocessor = ColumnTransformer([
    ('categ', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical),
    ('bool', 'passthrough', boolean),
    ('num', 'passthrough', numeric)
])

In [174]:
# Split the data
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
## now its time see the len of the variable ##
#x_train.shape=(414203, 12)
#x_test.shape=(103551, 12)
#y_train.shape=(414203,)
#y_test.shape=(103551,)

In [175]:
# Define Model
model = Pipeline(steps=[
                ('preprocessor', preprocessor),
                ('regressor',LinearRegression())
                 ])

In [176]:
## For these i will use the model called Decision tree. it is best for predictind the accurency ans..

model.fit(X_train, y_train) 

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categ',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['road_type', 'lighting',
                                                   'weather', 'time_of_day']),
                                                 ('bool', 'passthrough',
                                                  ['road_signs_present',
                                                   'public_road', 'holiday',
                                                   'school_season']),
                                                 ('num', 'passthrough',
                                                  ['num_lanes', 'curvature',
                                                   'speed_limit',
                                                   'num_reported_accidents'])])),
                ('regressor', LinearRegression())])

In [188]:
y_pred=model.predict(X_val)
y_pred

array([0.2061064 , 0.26770369, 0.20898674, ..., 0.47528472, 0.12337377,
       0.2277949 ])

In [179]:
model.score(X_val,y_val)

0.8041884067087808

In [189]:
model.predict(X_test)


array([0.36489615, 0.15728385, 0.20857281, ..., 0.24294978, 0.17136553,
       0.48520561])

In [190]:
# Calculate metrics
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
r2 = r2_score(y_val, y_pred)
print("RMSE: ", rmse)
print("R2: ", r2)

RMSE:  0.07353086258983137
R2:  0.8041884067087808


In [191]:
y_pred2 = model.predict(X_test)
Sample['accident_risk'] = y_pred2
Sample.to_csv('submission.csv', index=False)
print(Sample.head())

       id  accident_risk
0  517754       0.364896
1  517755       0.157284
2  517756       0.208573
3  517757       0.436379
4  517758       0.329172
